In [1]:
import pandas as pd
import numpy as np

In [2]:
accidents = pd.read_csv('../data/2015-2019_dataset_clusters.csv')

In [3]:
accidents.head()

,x,y,accident_id,year,address,severity,accident_type,severity_numeric,borough_geo,timestamp,date,hour,month_name,month,day,cluster_id
0,-74.105296,4.509792,4437952,2016,CL 80A-KR 1 SE 02,Injury,Crash,8,USME,2016-02-27 16:20:00+00:00,2016-02-27,16,Feb,2,Sat,0
1,-74.167225,4.631051,4472304,2017,AV AVENIDA CIUDAD DE CALI-CL 42 S 02,Injury,Run over,9,KENNEDY,2017-02-09 16:45:00+00:00,2017-02-09,16,Feb,2,Thu,1
2,-74.121790,4.603106,4512837,2018,AV AVENIDA PRIMERA DE MAYO-KR 50A 14,Injury,Crash,8,PUENTE ARANDA,2018-03-25 12:10:00+00:00,2018-03-25,12,Mar,3,Sun,1
3,-74.075332,4.607944,4437462,2016,KR 13-CL 18 02,Injury,Crash,8,SANTA FE,2016-02-22 08:55:00+00:00,2016-02-22,8,Feb,2,Mon,1
4,-74.064965,4.744182,4473374,2017,AV AVENIDA BOYACA-CL 160 02,Injury,Run over,9,SUBA,2017-02-19 13:00:00+00:00,2017-02-19,13,Feb,2,Sun,2


In [4]:
df_positive_2018 = accidents[pd.to_datetime(accidents.date).dt.year == 2018]

In [5]:
df_positive = df_positive_2018[['x','y','date','hour', 'borough_geo', 'address', 'cluster_id']].copy()

In [6]:
df_positive.date = pd.to_datetime(df_positive.date)

In [7]:
df_positive['day_of_year'] = df_positive.date.dt.dayofyear

In [8]:
df_positive['coords'] = df_positive[['x', 'y']].apply(lambda x: ';'.join((str(x['x']), str(x['y']))), axis = 1)

In [9]:
df_positive = df_positive[['hour', 'day_of_year','borough_geo', 'coords', 'address', 'cluster_id']].copy()

In [10]:
df_positive.head(2)

,hour,day_of_year,borough_geo,coords,address,cluster_id
2,12,84,PUENTE ARANDA,-74.12178986799995;4.6031063950000535,AV AVENIDA PRIMERA DE MAYO-KR 50A 14,1
9,7,44,BOSA,-74.17893912999993;4.62416029700006,KR 86-CL 53 S 2,1


In [11]:
df_positive = df_positive.reset_index(drop=True)

In [12]:
def generate_negative_sample(row):
    num_samples = 0
    while num_samples < 1:
        randcol = np.random.choice(np.array(['hour', 'day_of_year', 'coords']))
        replacement = df_positive.sample()
        negative_sample = row
        negative_sample[randcol] = replacement[randcol].values[0]
        if randcol == 'coords':
            negative_sample['borough_geo'] = replacement['borough_geo'].values[0]
            negative_sample['address']  = replacement['address'].values[0]
            negative_sample['cluster_id']  = replacement['cluster_id'].values[0]

        ncoords = negative_sample['coords']
        nhour = negative_sample['hour']
        nday = negative_sample['day_of_year']
        
        if df_positive.loc[(df_positive['coords'] == ncoords) & (df_positive['hour'] == nhour) & (df_positive['day_of_year'] == nday)].empty:
            yield negative_sample[['hour', 'day_of_year','coords','address', 'borough_geo', 'cluster_id']].values.tolist()
            num_samples += 1
        else:
            print('found coincidence', negative_sample[['hour', 'day_of_year','coords']].values.tolist())

In [13]:
my_list = [generate_negative_sample(df_positive.iloc[x]) for x in df_positive.index]

In [14]:
%%time
from itertools import chain
neg_list = list(chain.from_iterable(my_list))

']
found coincidence [19, 16, '-74.10164187099997;4.69244135200006']
found coincidence [9, 98, '-74.06668429999998;4.71029513000002']
found coincidence [19, 52, '-74.12278393899999;4.558443448000048']
found coincidence [7, 111, '-74.09369859599997;4.688895240000021']
found coincidence [22, 30, '-74.19036733899995;4.622555699000031']
found coincidence [7, 163, '-74.09562420699997;4.650374386000067']
found coincidence [20, 110, '-74.10881555199995;4.669605164000076']
found coincidence [6, 26, '-74.11672821099995;4.686298769000075']
found coincidence [12, 147, '-74.10036860699995;4.703752692000024']
found coincidence [12, 69, '-74.04039635;4.689502100000027']
found coincidence [8, 82, '-74.10290485999997;4.6908317500000285']
found coincidence [12, 24, '-74.10378029599997;4.600985306000041']
found coincidence [17, 146, '-74.10225851199993;4.669959096000071']
found coincidence [15, 3, '-74.11021922499998;4.619483583000034']
found coincidence [15, 113, '-74.07772209699993;4.665658441000062']

In [15]:
neg_df = pd.DataFrame.from_records(neg_list)

In [16]:
neg_df

,0,1,2,3,4,5
0,12,318,-74.12178986799995;4.6031063950000535,AV AVENIDA PRIMERA DE MAYO-KR 50A 14,PUENTE ARANDA,1
1,13,44,-74.17893912999993;4.62416029700006,KR 86-CL 53 S 2,BOSA,1
2,17,173,-74.12102002699999;4.721841823000034,CL 80-KR 114 99,ENGATIVA,10
3,9,92,-74.06570572199999;4.599420861000056,AV AVENIDA CIRCUNVALAR-CL 17 02,CANDELARIA,13
4,8,170,-74.09429402399995;4.600073951000071,CL 4B-KR 23A 28,LOS MARTIRES,1
...,...,...,...,...,...,...
33196,12,214,-74.11071803599998;4.708741003000057,CL 80-KR 98A 2,ENGATIVA,1
33197,16,72,-74.07638578699994;4.685922248000054,KR 68-CL 94 02,BARRIOS UNIDOS,1
33198,23,248,-74.06070190099996;4.6780774160000655,KR 30-CL 91 2,BARRIOS UNIDOS,1
33199,15,255,-74.08714144399994;4.6752217400000395,KR 68-CL 67DBIS 2,BARRIOS UNIDOS,1


In [17]:
neg_df.columns = ['hour', 'day_of_year', 'coords', 'address', 'borough', 'cluster_id']

In [18]:
neg_df['year'] = 2018

In [19]:
neg_df.shape

(33201, 7)

In [20]:
def compose_date(years, months=1, days=1, weeks=None, hours=None, minutes=None,
                 seconds=None, milliseconds=None, microseconds=None, nanoseconds=None):
    years = np.asarray(years) - 1970
    months = np.asarray(months) - 1
    days = np.asarray(days) - 1
    types = ('<M8[Y]', '<m8[M]', '<m8[D]', '<m8[W]', '<m8[h]',
             '<m8[m]', '<m8[s]', '<m8[ms]', '<m8[us]', '<m8[ns]')
    vals = (years, months, days, weeks, hours, minutes, seconds,
            milliseconds, microseconds, nanoseconds)
    return sum(np.asarray(v, dtype=t) for t, v in zip(types, vals)
               if v is not None)


neg_df['timestamp'] = compose_date(neg_df['year'], days=neg_df['day_of_year'], hours=neg_df['hour'], minutes=np.zeros(neg_df.shape[0]), seconds=np.zeros(neg_df.shape[0]))
neg_df['date'] = compose_date(neg_df['year'], days=neg_df['day_of_year'])


In [21]:
neg_df = neg_df.join(neg_df.coords.str.split(';',expand=True))

In [22]:
neg_df.rename(columns={0: "x", 1: "y"}, inplace=True)

In [23]:
neg_df

,hour,day_of_year,coords,address,borough,cluster_id,year,timestamp,date,x,y
0,12,318,-74.12178986799995;4.6031063950000535,AV AVENIDA PRIMERA DE MAYO-KR 50A 14,PUENTE ARANDA,1,2018,2018-11-14 12:00:00,2018-11-14,-74.12178986799995,4.6031063950000535
1,13,44,-74.17893912999993;4.62416029700006,KR 86-CL 53 S 2,BOSA,1,2018,2018-02-13 13:00:00,2018-02-13,-74.17893912999993,4.62416029700006
2,17,173,-74.12102002699999;4.721841823000034,CL 80-KR 114 99,ENGATIVA,10,2018,2018-06-22 17:00:00,2018-06-22,-74.12102002699999,4.721841823000034
3,9,92,-74.06570572199999;4.599420861000056,AV AVENIDA CIRCUNVALAR-CL 17 02,CANDELARIA,13,2018,2018-04-02 09:00:00,2018-04-02,-74.06570572199999,4.599420861000056
4,8,170,-74.09429402399995;4.600073951000071,CL 4B-KR 23A 28,LOS MARTIRES,1,2018,2018-06-19 08:00:00,2018-06-19,-74.09429402399995,4.600073951000071
...,...,...,...,...,...,...,...,...,...,...,...
33196,12,214,-74.11071803599998;4.708741003000057,CL 80-KR 98A 2,ENGATIVA,1,2018,2018-08-02 12:00:00,2018-08-02,-74.11071803599998,4.708741003000057
33197,16,72,-74.07638578699994;4.685922248000054,KR 68-CL 94 02,BARRIOS UNIDOS,1,2018,2018-03-13 16:00:00,2018-03-13,-74.07638578699994,4.685922248000054
33198,23,248,-74.06070190099996;4.6780774160000655,KR 30-CL 91 2,BARRIOS UNIDOS,1,2018,2018-09-05 23:00:00,2018-09-05,-74.06070190099996,4.6780774160000655
33199,15,255,-74.08714144399994;4.6752217400000395,KR 68-CL 67DBIS 2,BARRIOS UNIDOS,1,2018,2018-09-12 15:00:00,2018-09-12,-74.08714144399994,4.6752217400000395


In [24]:
neg_df['accident_id'] = ''
neg_df['severity'] = ''
neg_df['accident_type'] = ''
neg_df['severity_numeric'] = ''
neg_df['day_of_week'] = neg_df.timestamp.dt.dayofweek
neg_df['day'] = neg_df.timestamp.dt.day
neg_df['month'] = neg_df.timestamp.dt.month

In [25]:
neg_df.head()

,hour,day_of_year,coords,address,borough,cluster_id,year,timestamp,date,x,y,accident_id,severity,accident_type,severity_numeric,day_of_week,day,month
0,12,318,-74.12178986799995;4.6031063950000535,AV AVENIDA PRIMERA DE MAYO-KR 50A 14,PUENTE ARANDA,1,2018,2018-11-14 12:00:00,2018-11-14,-74.12178986799995,4.6031063950000535,,,,,2,14,11
1,13,44,-74.17893912999993;4.62416029700006,KR 86-CL 53 S 2,BOSA,1,2018,2018-02-13 13:00:00,2018-02-13,-74.17893912999993,4.62416029700006,,,,,1,13,2
2,17,173,-74.12102002699999;4.721841823000034,CL 80-KR 114 99,ENGATIVA,10,2018,2018-06-22 17:00:00,2018-06-22,-74.12102002699999,4.721841823000034,,,,,4,22,6
3,9,92,-74.06570572199999;4.599420861000056,AV AVENIDA CIRCUNVALAR-CL 17 02,CANDELARIA,13,2018,2018-04-02 09:00:00,2018-04-02,-74.06570572199999,4.599420861000056,,,,,0,2,4
4,8,170,-74.09429402399995;4.600073951000071,CL 4B-KR 23A 28,LOS MARTIRES,1,2018,2018-06-19 08:00:00,2018-06-19,-74.09429402399995,4.600073951000071,,,,,1,19,6


In [26]:
accidents.drop(columns=['accident_id', 'month_name'], inplace=True)

In [27]:
accidents_2018 = accidents[accidents.year == 2018]

In [29]:
accidents_2018.timestamp = pd.to_datetime(accidents_2018.timestamp, utc=True).dt.tz_localize(None)

In [30]:
accidents_2018['day'] = accidents_2018.timestamp.dt.day
accidents_2018['day_of_week'] = accidents_2018.timestamp.dt.dayofweek
accidents_2018['day_of_year'] = accidents_2018.timestamp.dt.dayofyear

In [31]:
accidents_2018.head(20)

,x,y,year,address,severity,accident_type,severity_numeric,borough_geo,timestamp,date,hour,month,day,cluster_id,day_of_week,day_of_year
2,-74.121790,4.603106,2018,AV AVENIDA PRIMERA DE MAYO-KR 50A 14,Injury,Crash,8,PUENTE ARANDA,2018-03-25 12:10:00,2018-03-25,12,3,25,1,6,84
9,-74.178939,4.624160,2018,KR 86-CL 53 S 2,Injury,Crash,8,BOSA,2018-02-13 07:45:00,2018-02-13,7,2,13,1,1,44
30,-74.121020,4.721842,2018,CL 80-KR 114 99,Material damage,Crash,1,ENGATIVA,2018-05-02 17:50:00,2018-05-02,17,5,2,10,2,122
36,-74.065706,4.599421,2018,AV AVENIDA CIRCUNVALAR-CL 17 02,Material damage,Crash,1,CANDELARIA,2018-04-26 09:00:00,2018-04-26,9,4,26,13,3,116
51,-74.094294,4.600074,2018,CL 4B-KR 23A 28,Injury,Run over,9,LOS MARTIRES,2018-06-19 12:00:00,2018-06-19,12,6,19,1,1,170
52,-74.079973,4.600711,2018,KR 12-CL 10 49,Material damage,Crash,1,SANTA FE,2018-06-18 14:55:00,2018-06-18,14,6,18,1,0,169
53,-74.069820,4.638032,2018,KR 17-CL 50 02,Injury,Occupant fall,7,TEUSAQUILLO,2018-06-22 10:40:00,2018-06-22,10,6,22,1,4,173
54,-74.077495,4.665175,2018,CL 68-KR 51 02,Material damage,Crash,1,BARRIOS UNIDOS,2018-07-18 10:30:00,2018-07-18,10,7,18,1,2,199
55,-74.179553,4.603938,2018,CL 59-KR 78D S 2,Material damage,Crash,1,BOSA,2018-07-06 07:15:00,2018-07-06,7,7,6,1,4,187
56,-74.058813,4.657149,2018,KR 11-CL 72 2,Material damage,Crash,1,CHAPINERO,2018-07-09 23:30:00,2018-07-09,23,7,9,1,0,190


In [32]:
accidents_2018 = accidents_2018[['x', 'y', 'timestamp', 'date', 'year', 'month',
       'day', 'day_of_year', 'day_of_week', 'hour', 'address', 'severity', 'accident_type',
       'severity_numeric', 'borough_geo','cluster_id']]

In [33]:
accidents_2018.rename(columns={"borough_geo": "borough"}, inplace=True)

In [34]:
accidents_2018['sample_type'] = 1

In [35]:
neg_df = neg_df[['x', 'y', 'timestamp', 'date', 'year', 'month',
       'day', 'day_of_year', 'day_of_week', 'hour', 'address', 'severity', 'accident_type',
       'severity_numeric', 'borough','cluster_id']]

In [36]:
neg_df['sample_type'] = 0

In [38]:
accidents_2018.shape

(33201, 17)

In [39]:
dataset = pd.concat([accidents_2018,neg_df])

In [40]:
dataset

,x,y,timestamp,date,year,month,day,day_of_year,day_of_week,hour,address,severity,accident_type,severity_numeric,borough,cluster_id,sample_type
2,-74.1218,4.60311,2018-03-25 12:10:00,2018-03-25,2018,3,25,84,6,12,AV AVENIDA PRIMERA DE MAYO-KR 50A 14,Injury,Crash,8,PUENTE ARANDA,1,1
9,-74.1789,4.62416,2018-02-13 07:45:00,2018-02-13,2018,2,13,44,1,7,KR 86-CL 53 S 2,Injury,Crash,8,BOSA,1,1
30,-74.121,4.72184,2018-05-02 17:50:00,2018-05-02,2018,5,2,122,2,17,CL 80-KR 114 99,Material damage,Crash,1,ENGATIVA,10,1
36,-74.0657,4.59942,2018-04-26 09:00:00,2018-04-26,2018,4,26,116,3,9,AV AVENIDA CIRCUNVALAR-CL 17 02,Material damage,Crash,1,CANDELARIA,13,1
51,-74.0943,4.60007,2018-06-19 12:00:00,2018-06-19,2018,6,19,170,1,12,CL 4B-KR 23A 28,Injury,Run over,9,LOS MARTIRES,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33196,-74.11071803599998,4.708741003000057,2018-08-02 12:00:00,2018-08-02 00:00:00,2018,8,2,214,3,12,CL 80-KR 98A 2,,,,ENGATIVA,1,0
33197,-74.07638578699994,4.685922248000054,2018-03-13 16:00:00,2018-03-13 00:00:00,2018,3,13,72,1,16,KR 68-CL 94 02,,,,BARRIOS UNIDOS,1,0
33198,-74.06070190099996,4.6780774160000655,2018-09-05 23:00:00,2018-09-05 00:00:00,2018,9,5,248,2,23,KR 30-CL 91 2,,,,BARRIOS UNIDOS,1,0
33199,-74.08714144399994,4.6752217400000395,2018-09-12 15:00:00,2018-09-12 00:00:00,2018,9,12,255,2,15,KR 68-CL 67DBIS 2,,,,BARRIOS UNIDOS,1,0


In [41]:
boroughs = pd.read_csv("../data/localidades.csv")

In [42]:
boroughs.borough.unique()

array(['USAQUEN', 'CHAPINERO', 'SANTA FE', 'SAN CRISTOBAL', 'USME',
       'TUNJUELITO', 'BOSA', 'KENNEDY', 'FONTIBON', 'ENGATIVA', 'SUBA',
       'BARRIOS UNIDOS', 'TEUSAQUILLO', 'LOS MARTIRES', 'ANTONIO NARIÑO',
       'PUENTE ARANDA', 'CANDELARIA', 'RAFAEL URIBE URIBE',
       'CIUDAD BOLIVAR', 'SUMAPAZ'], dtype=object)

In [43]:
boroughs.columns

Index(['id', 'borough', 'area_km2', 'population', 'population_density'], dtype='object')

In [44]:
# boroughs.replace({"Tunjuelito": 'TUNJUELITO', "Chapinero":'CHAPINERO', "Kennedy": 'KENNEDY', "Usaquén": 'USAQUEN', "BarriosUnidos":  'BARRIOS UNIDOS', "CiudadBolívar": 
# 'CIUDAD BOLIVAR', "Fontibón":  'FONTIBON', "PuenteAranda":  'PUENTE ARANDA', "Suba":  'SUBA', "Teusaquillo":
# 'TEUSAQUILLO', "SantaFe":  'SANTA FE', "Bosa":  'BOSA', "SanCristóbal":  'SAN CRISTOBAL', "Engativá":  'ENGATIVA', "RafaelUribeUribe": 
# 'RAFAEL URIBE URIBE', "AntonioNariño":  'ANTONIO NARIÑO', "LosMártires":  'LOS MARTIRES', "LaCandelaria":
# 'CANDELARIA', "Usme": 'USME', "Sumapaz":  'SUMAPAZ'}, inplace=True)

In [45]:
# boroughs.drop(columns = ['id'], inplace=True)

In [46]:
dataset = dataset.merge(boroughs, how='left', left_on='borough', right_on='borough')

In [47]:
dataset.head()

,x,y,timestamp,date,year,month,day,day_of_year,day_of_week,hour,...,severity,accident_type,severity_numeric,borough,cluster_id,sample_type,id,area_km2,population,population_density
0,-74.1218,4.60311,2018-03-25 12:10:00,2018-03-25,2018,3,25,84,6,12,...,Injury,Crash,8,PUENTE ARANDA,1,1,16,17.31,258287,14921.25
1,-74.1789,4.62416,2018-02-13 07:45:00,2018-02-13,2018,2,13,44,1,7,...,Injury,Crash,8,BOSA,1,1,7,23.93,673077,28126.91
2,-74.121,4.72184,2018-05-02 17:50:00,2018-05-02,2018,5,2,122,2,17,...,Material damage,Crash,1,ENGATIVA,10,1,10,35.88,887080,24723.52
3,-74.0657,4.59942,2018-04-26 09:00:00,2018-04-26,2018,4,26,116,3,9,...,Material damage,Crash,1,CANDELARIA,13,1,17,2.06,24088,11693.20
4,-74.0943,4.60007,2018-06-19 12:00:00,2018-06-19,2018,6,19,170,1,12,...,Injury,Run over,9,LOS MARTIRES,1,1,14,6.51,99119,15225.65


In [48]:
dataset.rename(columns={"id": "borough_id"}, inplace=True)

In [49]:
weather = pd.read_csv("../data/total.csv")

In [50]:
weather.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'time', 'summary', 'icon',
       'precipIntensity', 'precipProbability', 'temperature',
       'apparentTemperature', 'dewPoint', 'humidity', 'windSpeed',
       'cloudCover', 'uvIndex', 'visibility', 'windBearing', 'location',
       'windGust', 'pressure', 'precipType', 'ozone', 'precipAccumulation'],
      dtype='object')

In [51]:
weather.drop(columns=["Unnamed: 0", "Unnamed: 0.1"], inplace=True)

In [52]:
weather.head()

,time,summary,icon,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,windSpeed,cloudCover,uvIndex,visibility,windBearing,location,windGust,pressure,precipType,ozone,precipAccumulation
0,2014-07-15 05:00:00,Partly Cloudy,partly-cloudy-night,0.0,0.0,10.0,10.00,9.99,1.00,1.02,0.44,0.0,10.003,NaN,bosa,NaN,NaN,NaN,NaN,NaN
1,2014-07-15 06:00:00,Partly Cloudy,partly-cloudy-night,0.0,0.0,9.0,9.00,9.00,1.00,1.02,0.44,0.0,10.003,50.0,bosa,NaN,NaN,NaN,NaN,NaN
2,2014-07-15 07:00:00,Partly Cloudy,partly-cloudy-night,0.0,0.0,9.0,9.00,9.00,1.00,1.02,0.44,0.0,10.003,NaN,bosa,NaN,NaN,NaN,NaN,NaN
3,2014-07-15 08:00:00,Partly Cloudy,partly-cloudy-night,0.0,0.0,10.0,8.46,9.00,0.94,3.09,0.44,0.0,10.003,120.0,bosa,NaN,NaN,NaN,NaN,NaN
4,2014-07-15 09:00:00,Partly Cloudy,partly-cloudy-night,0.0,0.0,9.0,8.46,6.99,0.87,1.55,0.44,0.0,10.003,270.0,bosa,NaN,NaN,NaN,NaN,NaN


In [53]:
dataset['timestamp_floor'] =  dataset.timestamp.dt.round('H')

In [54]:
weather.time = pd.to_datetime(weather.time)

In [55]:
weather.location.unique()

array(['bosa', 'engativa', 'tunjuelito', 'fontibon', 'kennedy',
       'ciudadbolivar', 'la candelaria', 'puente aranda', 'rafael uribe',
       'sumapaz', 'antonio nariño', 'barrios unidos', 'chapinero',
       'los martires', 'san cristobal', 'santafe', 'suba', 'teusaquillo',
       'usaquen', 'usme'], dtype=object)

In [56]:
weather.replace({"tunjuelito": 'TUNJUELITO', "chapinero":'CHAPINERO', "kennedy": 'KENNEDY', "usaquen": 'USAQUEN', "barrios unidos":  'BARRIOS UNIDOS', "ciudadbolivar": 
'CIUDAD BOLIVAR', "fontibon":  'FONTIBON', "puente aranda":  'PUENTE ARANDA', "suba":  'SUBA', "teusaquillo":
'TEUSAQUILLO', "santafe":  'SANTA FE', "bosa":  'BOSA', "san cristobal":  'SAN CRISTOBAL', "engativa":  'ENGATIVA', "rafael uribe": 
'RAFAEL URIBE URIBE', "antonio nariño":  'ANTONIO NARIÑO', "los martires":  'LOS MARTIRES', "la candelaria":
'CANDELARIA', "usme": 'USME', "sumapaz":  'SUMAPAZ'}, inplace=True)

In [57]:
dataset = dataset.merge(weather, left_on=['borough' , 'timestamp_floor'], right_on=['location', 'time'])

In [58]:
dataset.columns

Index(['x', 'y', 'timestamp', 'date', 'year', 'month', 'day', 'day_of_year',
       'day_of_week', 'hour', 'address', 'severity', 'accident_type',
       'severity_numeric', 'borough', 'cluster_id', 'sample_type',
       'borough_id', 'area_km2', 'population', 'population_density',
       'timestamp_floor', 'time', 'summary', 'icon', 'precipIntensity',
       'precipProbability', 'temperature', 'apparentTemperature', 'dewPoint',
       'humidity', 'windSpeed', 'cloudCover', 'uvIndex', 'visibility',
       'windBearing', 'location', 'windGust', 'pressure', 'precipType',
       'ozone', 'precipAccumulation'],
      dtype='object')

In [59]:
dataset.drop(columns = ['timestamp_floor', 'location'], inplace=True)

In [60]:
dataset.rename(columns={"time": "timestamp_darksky"}, inplace=True)

In [61]:
dataset.to_csv('../data/dataset_2018_with_negatives.csv', index=None)

In [62]:
dataset

,x,y,timestamp,date,year,month,day,day_of_year,day_of_week,hour,...,windSpeed,cloudCover,uvIndex,visibility,windBearing,windGust,pressure,precipType,ozone,precipAccumulation
0,-74.1218,4.60311,2018-03-25 12:10:00,2018-03-25,2018,3,25,84,6,12,...,2.09,0.44,0.0,10.003,30.0,NaN,NaN,NaN,NaN,NaN
1,-74.10631202499997,4.6273950890000615,2018-03-25 12:00:00,2018-03-25 00:00:00,2018,3,25,84,6,12,...,2.09,0.44,0.0,10.003,30.0,NaN,NaN,NaN,NaN,NaN
2,-74.1789,4.62416,2018-02-13 07:45:00,2018-02-13,2018,2,13,44,1,7,...,1.02,0.44,0.0,10.003,NaN,NaN,NaN,NaN,NaN,NaN
3,-74.121,4.72184,2018-05-02 17:50:00,2018-05-02,2018,5,2,122,2,17,...,5.09,0.75,7.0,10.003,150.0,NaN,NaN,NaN,NaN,NaN
4,-74.1047,4.65641,2018-05-02 18:19:00,2018-05-02,2018,5,2,122,2,18,...,5.09,0.75,7.0,10.003,150.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66217,-74.10435691599997,4.652615037000032,2018-03-11 20:00:00,2018-03-11 00:00:00,2018,3,11,70,6,20,...,5.09,0.44,5.0,10.003,120.0,NaN,NaN,NaN,NaN,NaN
66218,-74.10566061099996,4.705324347000044,2018-12-04 22:00:00,2018-12-04 00:00:00,2018,12,4,338,1,22,...,1.77,0.88,0.0,16.093,307.0,3.15,1011.5,rain,241.6,NaN
66219,-74.06577121299993,4.755576805000032,2018-09-15 13:00:00,2018-09-15 00:00:00,2018,9,15,258,5,13,...,1.05,0.90,3.0,14.669,58.0,1.62,1015.7,rain,269.1,NaN
66220,-74.06070190099996,4.6780774160000655,2018-09-05 23:00:00,2018-09-05 00:00:00,2018,9,5,248,2,23,...,1.85,0.90,0.0,11.843,162.0,3.25,1014.3,rain,261.2,NaN
